# Train A Shape Classifier Model



In [ ]:
import json
import os

train_data_root = "../datasets/train"
test_data_root = "../datasets/test"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations (including resizing and normalization)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale (black and white images)
    transforms.Resize((64, 64)),  # Resize images to 64x64 pixels
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images (mean=0.5, std=0.5 for grayscale)
])

# Load the dataset
train_dataset = datasets.ImageFolder(root=train_data_root, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_root, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class names (optional)
print(f'Classes: {train_dataset.classes}')

# 2. Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        # self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 3)  # 3 classes: circle, triangle, rectangle
        
    def forward(self, x):
        x = F.relu(self.conv1(x))   # First Conv Layer
        x = F.max_pool2d(x, 2)      # Max Pooling
        x = F.relu(self.conv2(x))   # Second Conv Layer
        x = F.max_pool2d(x, 2)      # Max Pooling
        x = x.view(x.size(0), -1)   # Flatten
        x = F.relu(self.fc1(x))     # Fully Connected Layer 1
        x = self.fc2(x)             # Fully Connected Layer 2 (output)
        return x

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        # accuracy = 0 * correct / total
        accuracy = 100.0 * correct / total
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%')


# Train

In [ ]:
train_model(model, train_loader, criterion, optimizer, epochs=15)

# Test 

In [ ]:
def test(model, test_loader):
    """Print the Precision, Recall and F1-score for the trained model
    """

    total_true = []
    total_pred = []

    # Run through the entire test_loader and collect all the actual labels and predicted labels
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)                 
            _, predicted = torch.max(outputs, 1)    

            # Added to the list
            total_true.extend(labels.cpu().tolist())
            total_pred.extend(predicted.cpu().tolist())

    classes = test_loader.dataset.classes   
    classes_num = len(classes)

    # Prepare TP / FP / FN counters
    TP = [0] * classes_num
    FP = [0] * classes_num
    FN = [0] * classes_num

    # Iterate through each sample and update TP / FP / FN
    for y_true, y_pred in zip(total_true, total_pred):
        if y_true == y_pred:
            TP[y_true] += 1
        else:
            FP[y_pred] += 1
            FN[y_true] += 1

    # Calculate Precision / Recall / F1 for each class
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for idx, class_name in enumerate(classes):
        tp = TP[idx]
        fp = FP[idx]
        fn = FN[idx]
        support = tp + fn   # number of occurrences of each class in y_true

        if tp + fp > 0:
            precision = tp / (tp + fp)
        else:
            precision = 0.0

        if tp + fn > 0:
            recall = tp / (tp + fn)
        else:
            recall = 0.0

        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0.0

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        print(
            f"{class_name:10s} "
            f"Precision: {precision*100:5.2f}%  "
            f"Recall: {recall*100:5.2f}%  "
            f"F1: {f1*100:5.2f}%  "
            f"Support: {support}"
        )


In [ ]:
test(model, test_loader)

# Show Predictions


In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

def show_prediction(model, image):
    """Pass the image to the model and overlay the predicted shape and confidence on the input
    image and display it
    """
 
    # Increase the batch dimension
    image_tensor = image.unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)

    predicted_class = train_dataset.classes[predicted.item()]
    confidence = confidence.item() * 100

    # Convert the image tensor back to CPU and into a NumPy array for display
    shown_image = image.squeeze().cpu().numpy()

    plt.imshow(shown_image, cmap='gray')
    plt.title(f"Predicted: {predicted_class} {confidence:.1f}% ")
    plt.axis('off')
    plt.show()

In [ ]:
# Test with a picture
test_image, _ = test_dataset[0]
show_prediction(model, test_image)